In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train.head()

,filename,target_00,target_01,target_02,target_03,target_04,target_05,target_06,target_07,target_08,...,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22
0,8272,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,7833,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,8844,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,6586,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,7666,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
d = {"filename" : [],
     "text" : []}
path = "C:\\Users\\dell\\Desktop\\Data Engineering\\INE3\\ML_Comp\\challenge 5\\data\\data\\train"
cs = os.listdir(path)

for c in cs:
    fcs = path+"\\"+c
    fns = os.listdir(fcs)
    for fn in fns :
        f_path = path+"\\"+c+"\\"+fn

        text_f = open(f_path, "r").read().replace('\n', '')
        d['filename'].append(fn)
        d['text'].append(text_f)

d['filename']=list(map(lambda s: s.lstrip("0"), d["filename"]))
df = pd.DataFrame(d)
df = df.drop(df[df['filename']==''].index)
df['filename'] = df['filename'].astype(np.int64)
df

,filename,text
0,21,Augmentation mentoplasty using Mersilene mesh....
1,33,Multiple intracranial mucoceles associated wit...
2,52,Replacement of an aortic valve cusp after neon...
3,306,The value of indium 111 leukocyte scanning in ...
4,334,Febrile infants less than eight weeks old. Pre...
...,...,...
10428,9991,Afferent nipple valve malfunction caused by an...
10429,9992,Extracorporeal shock wave lithotripsy in combi...
10430,9993,Tamm-Horsfall autoantibodies in interstitial c...
10431,9995,Results of contemporary radical cystectomy for...


In [4]:
train_df = df.merge(train, on="filename", how='inner')
train_df.head(2)

,filename,text,target_00,target_01,target_02,target_03,target_04,target_05,target_06,target_07,...,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22
0,21,Augmentation mentoplasty using Mersilene mesh....,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,21,Augmentation mentoplasty using Mersilene mesh....,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
# Create and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df.text)
# Create the (X, Y) variables
X_train = tokenizer.texts_to_sequences(train_df.text)
X_train = pad_sequences(X_train, maxlen=400)
#Y_train = to_categorical(train_df.target)

In [6]:
X_train.shape

(10432, 400)

In [7]:
df1 = train_df.drop(['filename', 'text'], axis=1)
Y_train = df1.to_numpy()

In [8]:
Y_train.shape

(10432, 23)

In [19]:
from keras.models import Sequential                                    
from keras.layers import Flatten, Dense 
from keras.layers import Embedding
from keras.layers import LSTM

# Build the model
model = Sequential()
model.add(Embedding(100000, 128))
model.add(LSTM(128, dropout=0.2))
# Output layer has 23 units and uses `softmax`
model.add(Dense(23, activation="softmax"))
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Train the model
model.fit(X_train, Y_train,           
          batch_size=64, 
          epochs=10)

Epoch 1/10
163/163 [==============================] - 153s 931ms/step - loss: 6.4627 - accuracy: 0.0441
Epoch 2/10
163/163 [==============================] - 172s 1s/step - loss: 6.4575 - accuracy: 0.0361
Epoch 3/10
163/163 [==============================] - 187s 1s/step - loss: 6.5794 - accuracy: 0.0361
Epoch 4/10
163/163 [==============================] - 186s 1s/step - loss: 6.7097 - accuracy: 0.0361
Epoch 5/10
163/163 [==============================] - 206s 1s/step - loss: 6.8308 - accuracy: 0.0361
Epoch 6/10
163/163 [==============================] - 199s 1s/step - loss: 6.9339 - accuracy: 0.0361
Epoch 7/10
163/163 [==============================] - 218s 1s/step - loss: 7.0617 - accuracy: 0.0361
Epoch 8/10
163/163 [==============================] - 226s 1s/step - loss: 7.1004 - accuracy: 0.0361
Epoch 9/10
163/163 [==============================] - 209s 1s/step - loss: 7.2592 - accuracy: 0.0361
Epoch 10/10
163/163 [==============================] - 210s 1s/step - loss: 7.3226 - acc

In [21]:
d = {"filename" : [],
     "text" : []}
path = "C:\\Users\\dell\\Desktop\\Data Engineering\\INE3\\ML_Comp\\challenge 5\\data\\data\\test"
cs = os.listdir(path)

for fn in cs:
    f_path = path+"\\"+fn
    text_f = open(f_path, "r").read().replace('\n', '')
    d['filename'].append(fn)
    d['text'].append(text_f)

In [22]:
d['filename']=list(map(lambda s: s.lstrip("0"), d["filename"]))
test_df = pd.DataFrame(d)
import numpy as np
test_df['filename'] = test_df['filename'].astype(np.int64)
test_df

,filename,text
0,1,Cause of death in an emergency department A r...
1,10,Mistaken diagnoses in continuous muscle fiber ...
2,11,Haemophilus influenzae meningitis with prolong...
3,12,Dystrophin analysis in the differential diagno...
4,13,EEG correlation of improvement in hemolytic-ur...
...,...,...
7638,19988,Clinical utility of a position-monitoring cath...
7639,19989,Atracurium decay and the formation of laudanos...
7640,19990,The induction dose of propofol in infants 1-6 ...
7641,19992,Postoperative myocardial ischemia in patients ...


In [23]:
X_test = tokenizer.texts_to_sequences(test_df.text)
X_test = pad_sequences(X_test, maxlen=400)

X_test.shape

(7643, 400)

In [24]:
preds = np.round(model.predict(X_test),0).astype(int)
len(preds)

7643

In [25]:
filename = list(test_df['filename'].values)

In [26]:
submission = pd.DataFrame(preds, columns=['target_00', 'target_01',	'target_02', 'target_03',	'target_04',	
                             'target_05',	'target_06',	'target_07', 'target_08', 'target_09',
                             'target_10','target_11','target_12','target_13','target_14', 'target_15',
                             'target_16', 'target_17', 'target_18', 'target_19', 'target_20',
                             'target_21', 'target_22'])

In [27]:
submission['filename'] = filename

In [28]:
submission.to_csv('sub3.csv', index=False)

##### Score : 0.92765